In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'hard-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (enc_eta, enc_z)

In [ ]:
from ag_ep import *

train_prior(models, EUBO_init_eta_prior, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (93s),  symKL_DB_eta: 489.662,  symKL_DB_z: 503.363,  loss: -1744.509,  ess: 2.747,  kl_eta_ex: 759.323,  kl_eta_in: 191.983,  kl_z_ex: 169.241,  kl_z_in: 32.687
epoch: 1\250 (93s),  symKL_DB_eta: 388.689,  symKL_DB_z: 383.313,  loss: -1425.195,  ess: 3.029,  kl_eta_ex: 561.861,  kl_eta_in: 203.041,  kl_z_ex: 127.453,  kl_z_in: 27.208
epoch: 2\250 (93s),  symKL_DB_eta: 324.350,  symKL_DB_z: 290.872,  loss: -1237.121,  ess: 3.259,  kl_eta_ex: 448.937,  kl_eta_in: 228.902,  kl_z_ex: 113.763,  kl_z_in: 24.710
epoch: 3\250 (93s),  symKL_DB_eta: 291.651,  symKL_DB_z: 251.249,  loss: -1129.058,  ess: 3.349,  kl_eta_ex: 375.538,  kl_eta_in: 262.261,  kl_z_ex: 98.110,  kl_z_in: 22.388
epoch: 4\250 (93s),  symKL_DB_eta: 282.089,  symKL_DB_z: 228.777,  loss: -1069.017,  ess: 3.415,  kl_eta_ex: 328.353,  kl_eta_in: 303.252,  kl_z_ex: 79.985,  kl_z_in: 19.731
epoch: 5\250 (93s),  symKL_DB_eta: 266.768,  symKL_DB_z: 198.563,  loss: -1026.624,  ess: 3.556,  kl_eta_ex: 301.827,  kl_eta_i

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)